In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector.base import BaseExampleSelector


chat = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.8,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

## FewShotPromptTemplate


In [3]:
examples = [
    {
        "question": "Tell me about the movie *Inception*.",
        "answer": """
        Here is the information about the movie *Inception*:

        **Director:** Christopher Nolan  
        **Main Cast:** Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen Page, Tom Hardy  
        **Budget:** $160 million  
        **Box Office Revenue:** $836.8 million  
        **Genre:** Science Fiction, Thriller, Action  
        **Synopsis:** A skilled thief, Dom Cobb, who steals secrets through dream invasion, is offered a chance to erase his criminal history if he can plant an idea in someone's mind using a process called 'Inception.'  
        """,
    },
    {
        "question": "What can you tell me about *Parasite*?",
        "answer": """
        Here is the information about the movie *Parasite*:

        **Director:** Bong Joon-ho  
        **Main Cast:** Song Kang-ho, Lee Sun-kyun, Cho Yeo-jeong, Choi Woo-shik, Park So-dam  
        **Budget:** $11 million  
        **Box Office Revenue:** $263 million  
        **Genre:** Thriller, Drama, Dark Comedy  
        **Synopsis:** The story follows the Kim family, who scheme to become employed by a wealthy family, the Parks, by infiltrating their household through deception, leading to unexpected and tragic consequences.  
        """,
    },
]

In [15]:
class RandomExampleSeletor(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice

        print("input_variables:", input_variables)
        return [choice(self.examples)]

In [7]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate


example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

example_selector = RandomExampleSeletor(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: Tell me about the movie {movie}",
    input_variables=["movie"],
)

prompt.format(movie="Harry Potter and the Sorcerer's Stone")

"Human: Tell me about the movie *Inception*.\nAI: \n        Here is the information about the movie *Inception*:\n\n        **Director:** Christopher Nolan  \n        **Main Cast:** Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen Page, Tom Hardy  \n        **Budget:** $160 million  \n        **Box Office Revenue:** $836.8 million  \n        **Genre:** Science Fiction, Thriller, Action  \n        **Synopsis:** A skilled thief, Dom Cobb, who steals secrets through dream invasion, is offered a chance to erase his criminal history if he can plant an idea in someone's mind using a process called 'Inception.'  \n        \n\nHuman: Tell me about the movie Harry Potter and the Sorcerer's Stone"

In [8]:
chain = prompt | chat
chain.invoke({"movie": "Harry Potter and the Sorcerer's Stone"})

AI:
        Here is the information about the movie *Harry Potter and the Sorcerer's Stone*:

        **Director:** Chris Columbus  
        **Main Cast:** Daniel Radcliffe, Rupert Grint, Emma Watson, Robbie Coltrane, Richard Harris  
        **Budget:** $125 million  
        **Box Office Revenue:** $974.8 million  
        **Genre:** Fantasy, Adventure  
        **Synopsis:** The story follows a young boy named Harry Potter who discovers he is a wizard and attends Hogwarts School of Witchcraft and Wizardry, where he uncovers a dark secret about his past and battles the dark wizard Voldemort.

AIMessageChunk(content="AI:\n        Here is the information about the movie *Harry Potter and the Sorcerer's Stone*:\n\n        **Director:** Chris Columbus  \n        **Main Cast:** Daniel Radcliffe, Rupert Grint, Emma Watson, Robbie Coltrane, Richard Harris  \n        **Budget:** $125 million  \n        **Box Office Revenue:** $974.8 million  \n        **Genre:** Fantasy, Adventure  \n        **Synopsis:** The story follows a young boy named Harry Potter who discovers he is a wizard and attends Hogwarts School of Witchcraft and Wizardry, where he uncovers a dark secret about his past and battles the dark wizard Voldemort.")

### Load prompt


In [30]:
from langchain.prompts import load_prompt

prompt = load_prompt("./prompt.yaml")
prompt.format(movie="Wicked")

'Tell me about the movie Wicked'

In [39]:
prompt = load_prompt("./prompt.json")
prompt.format(movie="Wicked")

# load_prompt는 파일의 최상위 레벨에서 _type 키를 사용해 어떤 종류의 프롬프트를 로드할지 결정한다
# "_type" 키가 없으면 : No `_type` key found, defaulting to `prompt`.
# 키 이름은 정해져 있는 듯 (e.g "template" -> KeyError: 'example_prompt')

"Human: Tell me about the movie *Inception*.\nAI: **Director:** Christopher Nolan\n**Main Cast:** Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen Page, Tom Hardy\n**Budget:** $160 million\n**Box Office Revenue:** $836.8 million\n**Genre:** Science Fiction, Thriller, Action\n**Synopsis:** A skilled thief, Dom Cobb, who steals secrets through dream invasion, is offered a chance to erase his criminal history if he can plant an idea in someone's mind using a process called 'Inception.'\n\nHuman: What can you tell me about *Parasite*?\nAI: **Director:** Bong Joon-ho\n**Main Cast:** Song Kang-ho, Lee Sun-kyun, Cho Yeo-jeong, Choi Woo-shik, Park So-dam\n**Budget:** $11 million\n**Box Office Revenue:** $263 million\n**Genre:** Thriller, Drama, Dark Comedy\n**Synopsis:** The story follows the Kim family, who scheme to become employed by a wealthy family, the Parks, by infiltrating their household through deception, leading to unexpected and tragic consequences.\n\nHuman: Tell me about the movie 

In [37]:
chain = prompt | chat
response = chain.invoke({"movie": "Wicked"})

print(response.content)

AI: **Wicked** is actually a popular Broadway musical based on the novel *Wicked: The Life and Times of the Wicked Witch of the West* by Gregory Maguire. The musical, with music and lyrics by Stephen Schwartz, tells the untold story of the witches of Oz, focusing on the friendship between Glinda the Good Witch and Elphaba, the Wicked Witch of the West. The story explores themes of friendship, acceptance, and the nature of good and evil.AI: **Wicked** is actually a popular Broadway musical based on the novel *Wicked: The Life and Times of the Wicked Witch of the West* by Gregory Maguire. The musical, with music and lyrics by Stephen Schwartz, tells the untold story of the witches of Oz, focusing on the friendship between Glinda the Good Witch and Elphaba, the Wicked Witch of the West. The story explores themes of friendship, acceptance, and the nature of good and evil.


In [ ]:
# "suffix": "Human: Tell me about the movie {movie}\nAI: Please respond with the same format as the examples above.",
prompt = load_prompt("./prompt.json")

chain = prompt | chat
response = chain.invoke({"movie": "Wicked"})

print(response.content)  # 개봉한지 얼마 안돼서 반영 못한듯 ㅋㅋ

AI: I'm sorry, but there is no movie titled "Wicked." However, "Wicked" is a popular Broadway musical based on the novel "Wicked: The Life and Times of the Wicked Witch of the West" by Gregory Maguire. Would you like more information on the musical instead?AI: I'm sorry, but there is no movie titled "Wicked." However, "Wicked" is a popular Broadway musical based on the novel "Wicked: The Life and Times of the Wicked Witch of the West" by Gregory Maguire. Would you like more information on the musical instead?


In [ ]:
# "suffix": "Human: Tell me about the movie {movie}"
prompt = load_prompt("./prompt.json")

chain = prompt | chat
response = chain.invoke({"movie": "Iron Man"})

print(response.content)

AI: **Director:** Jon Favreau
**Main Cast:** Robert Downey Jr., Gwyneth Paltrow, Jeff Bridges, Terrence Howard
**Budget:** $140 million
**Box Office Revenue:** $585.2 million
**Genre:** Action, Sci-Fi
**Synopsis:** Billionaire industrialist Tony Stark, who is captured and forced to build a devastating weapon, instead creates a suit of armor to save his own life and escape. He later becomes the superhero Iron Man, using his technology to fight against evil.AI: **Director:** Jon Favreau
**Main Cast:** Robert Downey Jr., Gwyneth Paltrow, Jeff Bridges, Terrence Howard
**Budget:** $140 million
**Box Office Revenue:** $585.2 million
**Genre:** Action, Sci-Fi
**Synopsis:** Billionaire industrialist Tony Stark, who is captured and forced to build a devastating weapon, instead creates a suit of armor to save his own life and escape. He later becomes the superhero Iron Man, using his technology to fight against evil.


## FewShotChatMessagePromptTemplate


In [9]:
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate

In [13]:
examples = [
    {
        "movie": "*Inception*.",
        "answer": """
        Here is the information about the movie *Inception*:

        **Director:** Christopher Nolan  
        **Main Cast:** Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen Page, Tom Hardy  
        **Budget:** $160 million  
        **Box Office Revenue:** $836.8 million  
        **Genre:** Science Fiction, Thriller, Action  
        **Synopsis:** A skilled thief, Dom Cobb, who steals secrets through dream invasion, is offered a chance to erase his criminal history if he can plant an idea in someone's mind using a process called 'Inception.'  
        """,
    },
    {
        "movie": "*Parasite*",
        "answer": """
        Here is the information about the movie *Parasite*:

        **Director:** Bong Joon-ho  
        **Main Cast:** Song Kang-ho, Lee Sun-kyun, Cho Yeo-jeong, Choi Woo-shik, Park So-dam  
        **Budget:** $11 million  
        **Box Office Revenue:** $263 million  
        **Genre:** Thriller, Drama, Dark Comedy  
        **Synopsis:** The story follows the Kim family, who scheme to become employed by a wealthy family, the Parks, by infiltrating their household through deception, leading to unexpected and tragic consequences.  
        """,
    },
]

In [16]:
example_selector = RandomExampleSeletor(examples=examples)

# example prompt
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "Tell me about the movie {movie}"),
        ("ai", "{answer}"),
    ]
)

# FewShotChatMessagePromptTemplate
prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
)

# final prompt
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a well-known movie youtuber"),
        prompt,
        # example_prompt,
        ("human", "What can you tell me about {movie}?"),
    ]
)

In [21]:
final_prompt.format(movie="Pirates of the Caribbean")

input_variables: {}


'System: You are a well-known movie youtuber\nHuman: Tell me about the movie *Parasite*\nAI: \n        Here is the information about the movie *Parasite*:\n\n        **Director:** Bong Joon-ho  \n        **Main Cast:** Song Kang-ho, Lee Sun-kyun, Cho Yeo-jeong, Choi Woo-shik, Park So-dam  \n        **Budget:** $11 million  \n        **Box Office Revenue:** $263 million  \n        **Genre:** Thriller, Drama, Dark Comedy  \n        **Synopsis:** The story follows the Kim family, who scheme to become employed by a wealthy family, the Parks, by infiltrating their household through deception, leading to unexpected and tragic consequences.  \n        \nHuman: What can you tell me about Pirates of the Caribbean?'

In [17]:
chain = final_prompt | chat
chain.invoke({"movie": "Harry Potter and the Sorcerer's Stone"})

input_variables: {}

        **Title:** Harry Potter and the Sorcerer's Stone (known as *Harry Potter and the Philosopher's Stone* outside the US)  
        **Director:** Chris Columbus  
        **Main Cast:** Daniel Radcliffe, Rupert Grint, Emma Watson, Richard Harris, Maggie Smith, Alan Rickman  
        **Budget:** $125 million  
        **Box Office Revenue:** $974.8 million  
        **Genre:** Fantasy, Adventure  
        **Synopsis:** The film follows the story of a young boy named Harry Potter who discovers he is a wizard and is invited to attend Hogwarts School of Witchcraft and Wizardry. Along with his new friends Ron and Hermione, Harry learns about his magical abilities and uncovers a dark secret about his past. Together, they must confront the dark wizard Voldemort and prevent him from returning to power.  
           

AIMessageChunk(content="\n        **Title:** Harry Potter and the Sorcerer's Stone (known as *Harry Potter and the Philosopher's Stone* outside the US)  \n        **Director:** Chris Columbus  \n        **Main Cast:** Daniel Radcliffe, Rupert Grint, Emma Watson, Richard Harris, Maggie Smith, Alan Rickman  \n        **Budget:** $125 million  \n        **Box Office Revenue:** $974.8 million  \n        **Genre:** Fantasy, Adventure  \n        **Synopsis:** The film follows the story of a young boy named Harry Potter who discovers he is a wizard and is invited to attend Hogwarts School of Witchcraft and Wizardry. Along with his new friends Ron and Hermione, Harry learns about his magical abilities and uncovers a dark secret about his past. Together, they must confront the dark wizard Voldemort and prevent him from returning to power.  \n           ")